In [87]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import json
import pandas as pd
import warnings
import time 
import platform
import pymysql
import pandas as pd

conn = pymysql.connect(host="localhost",user="root",password="qkqh1234!",database="finalproject")

cur = conn.cursor()

sql = "SELECT bus_id,restaurant_name FROM restaurant"

cur.execute(sql)

data = cur.fetchall()

conn.close()

places = pd.DataFrame(data)


headers = {'Content-Type':"application/json"
    ,"accept":"*/*",'accept-encoding':'gzip, deflate, br','accept-language':"ko","User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"}

if "mac" in platform.platform():
    driver = webdriver.Chrome("./chromedriver")
else:
    driver = webdriver.Chrome("./chromedriver.exe")


driver.get("http://localhost:3000/login")

for i in range(6,7):
    driver.find_element(By.CSS_SELECTOR,
    "#root > div > div > div.SignUp_loginInfo__IKYx6 > a:nth-child(1)").click()

    driver.find_element(By.CSS_SELECTOR,
    "#root > div > div > div.SignUp_content__WR\+nJ > form > input:nth-child(2)").send_keys(f"gun_babo_{i}")

    driver.find_element(By.CSS_SELECTOR,
    "#root > div > div > div.SignUp_content__WR\+nJ > form > input:nth-child(6)").send_keys(f"gunkang{i}@naver.com")


    driver.find_element(By.CSS_SELECTOR,
    "#root > div > div > div.SignUp_content__WR\+nJ > form > input:nth-child(11)").send_keys("12345678")
    driver.find_element(By.CSS_SELECTOR,
    "#root > div > div > div.SignUp_content__WR\+nJ > form > input:nth-child(16)").send_keys("12345678")

    driver.find_element(By.CSS_SELECTOR,
    "#root > div > div > div.SignUp_content__WR\+nJ > form > div > button").click()

    time.sleep(0.2)
    driver.find_element(By.CSS_SELECTOR,
    "#root > div > div > div.Welcome_buttons__4jqHU > button:nth-child(1)").click()

    # 로그인 수행
    driver.find_element(By.CSS_SELECTOR,
    "#root > div > div > form > input:nth-child(2)").send_keys(f"gunkang{i}@naver.com")
    driver.find_element(By.CSS_SELECTOR,
    "#root > div > div > form > input:nth-child(6)").send_keys("12345678")

    driver.find_element(By.CSS_SELECTOR,
    "#root > div > div > form > div > button").click()
    
    time.sleep(0.3)
    # 마이페이지 -> 메인 페이지
    driver.find_element(By.CSS_SELECTOR, 
    "#root > div > div.Navbar_container__yPJt0 > div:nth-child(1) > a").click()

    driver.find_element(By.CSS_SELECTOR, 
    "#root > div > div > div.Header_background__aIS43 > div.Header_search__78Y2t > form > input").send_keys(places.iloc[0,1])
    driver.find_element(By.CSS_SELECTOR, 
    "#root > div > div > div.Header_background__aIS43 > div.Header_search__78Y2t > form > button").click()

    
    driver.find_element(By.CSS_SELECTOR,
    "#root > div > div.ListCard_wrapper__Gkssx > a > div > img").click()

    driver.find_element(By.CSS_SELECTOR,
    "#res-reviews > a").click()

    driver.find_element(By.CSS_SELECTOR,
    "#title").send_keys("hi")
    
    driver.find_element(By.CSS_SELECTOR,
    "#comment").send_keys("hihi44545454545656565656")

    driver.find_element(By.CSS_SELECTOR,
    "#root > div > div.ReviewWrite_container__7s5NG > form > div:nth-child(7) > button").click()



    


# driver.close()

# # 로그인 페이지 로드
# driver.get("http://localhost:3000/login")
# time.sleep(0.2)

# # 로그인 수행
# driver.find_element(By.CSS_SELECTOR,"#root > div > div > form > input:nth-child(2)").send_keys(f"gunkang{i}@naver.com")
# driver.find_element(By.CSS_SELECTOR,"#root > div > div > form > input:nth-child(6)").send_keys(f"12345678")
# driver.find_element(By.CSS_SELECTOR,"#root > div > div > form > div > button").click()
# time.sleep(0.2)





/var/folders/d1/5yg0q8_x0_s4wn9j6q1k_4n40000gn/T/ipykernel_1884/3502858759.py:31: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver")


In [33]:
places.iloc[0,1]

'낙원갈비집 평택 합정점'

In [ ]:
url = "https://pcmap-api.place.naver.com/graphql"

headers = {'Content-Type':"application/json"
    ,"accept":"*/*",'accept-encoding':'gzip, deflate, br','accept-language':"ko","User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"}

payload = {"operationName" : "getVisitorReviews",
"query" : "query getVisitorReviews($input: VisitorReviewsInput) {\n  visitorReviews(input: $input) {\n    items {\n      id\n      rating\n      author {\n        id\n        nickname\n        from\n        imageUrl\n        objectId\n        url\n        review {\n          totalCount\n          imageCount\n          avgRating\n          __typename\n        }\n        theme {\n          totalCount\n          __typename\n        }\n        __typename\n      }\n      body\n      thumbnail\n      media {\n        type\n        thumbnail\n        class\n        __typename\n      }\n      tags\n      status\n      visitCount\n      viewCount\n      visited\n      created\n      reply {\n        editUrl\n        body\n        editedBy\n        created\n        replyTitle\n        __typename\n      }\n      originType\n      item {\n        name\n        code\n        options\n        __typename\n      }\n      language\n      highlightOffsets\n      apolloCacheId\n      translatedText\n      businessName\n      showBookingItemName\n      showBookingItemOptions\n      bookingItemName\n      bookingItemOptions\n      votedKeywords {\n        code\n        iconUrl\n        iconCode\n        displayName\n        __typename\n      }\n      userIdno\n      isFollowing\n      followerCount\n      followRequested\n      loginIdno\n      receiptInfoUrl\n      __typename\n    }\n    starDistribution {\n      score\n      count\n      __typename\n    }\n    hideProductSelectBox\n    total\n    showRecommendationSort\n    itemReviewStats {\n      score\n      count\n      itemId\n      starDistribution {\n        score\n        count\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n",

"variables" : {
"input" : {
    "businessId" : "11802387",
"businessType": "restaurant"
},
}
}

res = json.loads(requests.post(headers=headers, url=url, data=json.dumps(payload)).text)

res

In [80]:
print(res['data']['visitorReviews']['items'][0].keys())
data = res['data']['visitorReviews']['items']

dict_keys(['id', 'rating', 'author', 'body', 'thumbnail', 'media', 'tags', 'status', 'visitCount', 'viewCount', 'visited', 'created', 'reply', 'originType', 'item', 'language', 'highlightOffsets', 'apolloCacheId', 'translatedText', 'businessName', 'showBookingItemName', 'showBookingItemOptions', 'bookingItemName', 'bookingItemOptions', 'votedKeywords', 'userIdno', 'isFollowing', 'followerCount', 'followRequested', 'loginIdno', 'receiptInfoUrl', '__typename'])


In [81]:
for b in data:
    print(b['body'])




잘 먹었습니다.
굿

굿



